## Setup

In [ ]:
! pip install hpelm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 8.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import hpelm  # Importing HPELM

#### Load_dataset

In [ ]:
data = load_digits()
X, y = data.data, data.target

## Pre-processing

In [ ]:
# One-hot encode the target
ohe = OneHotEncoder(sparse_output=False)
y_onehot = ohe.fit_transform(y.reshape(-1, 1))

# Split dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


## ELM-model training

In [ ]:
class ELM:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.W = np.random.randn(self.hidden_size, self.input_size)  # Random weights
        self.b = np.random.randn(self.hidden_size, 1)  # Random bias

    def _hidden_layer_output(self, X):
        return np.tanh(np.dot(self.W, X.T) + self.b)

    def train(self, X, Y):
        H = self._hidden_layer_output(X)
        H_T = H.T
        self.beta = np.dot(np.linalg.pinv(H_T), Y)  # Moore-Penrose Pseudo-inverse

    def predict(self, X):
        H = self._hidden_layer_output(X)
        return np.dot(H.T, self.beta)


### Custom model

#### Train Custom ELM

In [ ]:
start_time = time.time()
elm = ELM(input_size=X_train.shape[1], hidden_size=100, output_size=y_train.shape[1])
elm.train(X_train, y_train)
custom_elm_time = time.time() - start_time

#### Predict using Custom ELM

In [ ]:
y_pred_custom = elm.predict(X_test)
y_pred_labels_custom = np.argmax(y_pred_custom, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

#### Evaluate Custom ELM

In [ ]:
custom_elm_acc = accuracy_score(y_test_labels, y_pred_labels_custom)

### HPELM-(Hierarchical ELM)

In [ ]:
start_time = time.time()
hp_elm = hpelm.ELM(X_train.shape[1], y_train.shape[1])
hp_elm.add_neurons(100, "tanh")
hp_elm.train(X_train, y_train, "c")
hpelm_time = time.time() - start_time

#### Predict using HPELM


In [ ]:
y_pred_hpelm = hp_elm.predict(X_test)
y_pred_labels_hpelm = np.argmax(y_pred_hpelm, axis=1)

#### Evaluate HPELM

In [ ]:
hpelm_acc = accuracy_score(y_test_labels, y_pred_labels_hpelm)

## Model Summary

In [ ]:
# ---------------- Comparison Summary ---------------- #
print("Custom ELM Accuracy:", custom_elm_acc)
print("HPELM Accuracy:", hpelm_acc)
print("Custom ELM Training Time:", custom_elm_time)
print("HPELM Training Time:", hpelm_time)
print("Classification Report (Custom ELM):\n", classification_report(y_test_labels, y_pred_labels_custom))
print("Classification Report (HPELM):\n", classification_report(y_test_labels, y_pred_labels_hpelm))
print("Confusion Matrix (Custom ELM):\n", confusion_matrix(y_test_labels, y_pred_labels_custom))
print("Confusion Matrix (HPELM):\n", confusion_matrix(y_test_labels, y_pred_labels_hpelm))


Custom ELM Accuracy: 0.9277777777777778
HPELM Accuracy: 0.9222222222222223
Custom ELM Training Time: 0.09097146987915039
HPELM Training Time: 0.11864566802978516
Classification Report (Custom ELM):
               precision    recall  f1-score   support

           0       0.91      0.97      0.94        33
           1       0.96      0.93      0.95        28
           2       0.92      1.00      0.96        33
           3       0.94      0.94      0.94        34
           4       1.00      0.93      0.97        46
           5       0.90      0.96      0.93        47
           6       1.00      0.94      0.97        35
           7       0.89      0.91      0.90        34
           8       0.89      0.80      0.84        30
           9       0.88      0.88      0.88        40

    accuracy                           0.93       360
   macro avg       0.93      0.93      0.93       360
weighted avg       0.93      0.93      0.93       360

Classification Report (HPELM):
           